In [1]:
%pip install scikit-learn
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load raw dataset

import pandas as pd

sample = pd.read_csv('../data/raw/sample_submission.csv')
df_test = pd.read_csv('../data/raw/test.csv')
df_train = pd.read_csv('../data/raw/train.csv')

df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
# Preprocess features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_cols = [col for col in df_train.columns if col.startswith('var_')]
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])

In [4]:
# Show preprocessed features

df_train.head(1)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.53643,-0.334926,0.608751,...,0.263374,-1.149158,0.817469,-0.411013,0.168705,-1.578117,1.022131,-0.373968,-1.026398,0.214135


In [5]:
# Train model

import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Define LightGBM parameters
lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "boosting": 'gbdt',
    "max_depth": -1,
    "num_leaves": 31,
    "learning_rate": 0.01,
    "bagging_freq": 5,
    "bagging_fraction": 0.2,
    "feature_fraction": 0.1,
    "min_data_in_leaf": 100,
    "tree_learner": "data",
    "boost_from_average": "false",
    "verbosity": 2
}

# Split data
X = df_train.drop(['ID_code', 'target'], axis=1)
y = df_train['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=33)

# Train model
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
lgb_model = lgb.train(lgb_params, train_data, valid_sets=[val_data], num_boost_round=10000, 
                      callbacks = [
                          lgb.early_stopping(stopping_rounds=200)
                      ])

[LightGBM] [Info] Number of positive: 12153, number of negative: 107847
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000087 seconds, init for row-wise cost 0.026072 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 200
[LightGBM] [Debug] Re-bagging, using 24041 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
Training until validation scores don't improve for 200 rounds
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree 

#### LightGBM gave a validation set AUC of 0.89523